In [36]:
import gradio as gr
import ollama
from scenarios import scenarios,MAX_QUESTIONS,MAX_HINTS,tutorial_questions

def get_empty_state():
    return {"question_history": [], "used_answer": False, "hint_count": 0, "tutorial_step": 0}

scenario_states = [get_empty_state() for _ in scenarios]

def reset_game(idx):
    scenario_states[idx] = get_empty_state()
    return [], "", MAX_QUESTIONS, MAX_HINTS, idx

def handle_question(user_question, user_answer, request_hint, history, idx):
    state = scenario_states[idx]
    scenario = scenarios[idx]["scenario"]
    truth = scenarios[idx]["truth"]

    if state["used_answer"]:
        if request_hint:
            history.append({"role": "user", "content": "💡 힌트 요청"})
            history.append({"role": "assistant", "content": "이미 정답을 제출했습니다. 게임이 종료되었습니다. 홈으로 돌아가 새 시나리오를 선택하세요."})
        elif user_question.strip():
            history.append({"role": "user", "content": user_question})
            history.append({"role": "assistant", "content": "이미 정답을 제출했습니다. 게임이 종료되었습니다. 홈으로 돌아가 새 시나리오를 선택하세요."})
        elif user_answer.strip():
            history.append({"role": "user", "content": f"✅ 정답 제출: {user_answer}"})
            history.append({"role": "assistant", "content": "이미 정답을 제출했습니다. 게임이 종료되었습니다. 홈으로 돌아가 새 시나리오를 선택하세요."})
        return history, "", 0, MAX_HINTS - state["hint_count"], idx,""

    # 튜토리얼
    if idx == 0 and not user_answer and not request_hint:
        step = state.get("tutorial_step", 0)
        
        if step < len(tutorial_questions) and user_question.strip() == tutorial_questions[step]:
            # 사용자 질문 & 응답 추가
            history.append({"role": "user", "content": user_question})
            history.append({"role": "assistant", "content": "예"})
            state["tutorial_step"] = step + 1
            
            # 다음 튜토리얼 안내 메시지 (새로운 말풍선으로)
            if state["tutorial_step"] < len(tutorial_questions):
                next_q = tutorial_questions[state["tutorial_step"]]
                history.append({"role": "assistant", "content": 
                    f"\n\n이렇게 질문해 보세요!\n{state['tutorial_step']+1}️⃣ {next_q}"})
            else:
                history.append({"role": "assistant", "content": 
                    "\n\n 이제 정답에 가까워진 것 같습니다!\n\n 정답을 입력해보세요.\n남자는 진짜 바다거북스프를 먹고 전에 먹은 것이 시체로 끓인 스프란 것을 깨닫고 목숨을 끊었다."})
            
            state["question_history"].append(user_question)
            return history, "", MAX_QUESTIONS - len(state["question_history"]), MAX_HINTS - state["hint_count"], idx,""
            
    # 힌트 요청 처리
    if request_hint:
        if state["hint_count"] >= MAX_HINTS:
            history.append({"role": "user", "content": "💡 힌트 요청"})  # 사용자 메시지
            history.append({"role": "assistant", "content": "힌트 기회를 모두 사용했습니다."})  # 어시스턴트 메시지
            return history, "", MAX_QUESTIONS - len(state["question_history"]), MAX_HINTS - state["hint_count"], idx,""
            #return history, "힌트 기회를 모두 사용했습니다.", MAX_QUESTIONS - len(state["question_history"]), MAX_HINTS - state["hint_count"], idx

        hint_prompt = (
            "너는 반드시 아래 시나리오와 정답만을 참고해서, 정답을 직접적으로 노출하지 않으면서도 사용자가 진실에 가까워지도록 돕는 의미 있는 단서를 한 문장으로 생성하는 역할이야.\n"
            
            "### 힌트 생성 원칙 ###\n"
            "- 다른 시나리오나 외부 정보는 절대 참고하지 마.\n"
            "- 힌트는 너무 추상적이거나 전혀 관련 없는 내용이 아니어야 하며, 반드시 시나리오의 맥락과 연결되어야 해.\n"
            "- 정답의 핵심 이유와 단어는 직접 언급하면 안돼.\n"
            "- 주변 맥락이나 물리적 조건, 상황적 제약 등만 암시해.\n"
            
            "### 출력 원칙 ###\n"
            "힌트는 50글자 이내의 한 문장으로만 작성해야 하며, 반드시 그 한 문장만 출력해. 어떤 설명도 덧붙이지 말고 접두사도 붙이지 마."
        )
        messages = [
            {"role": "system", "content": hint_prompt},
            {"role": "user", "content": f"시나리오: {scenario}\n정답: {truth}"}
        ]
        res = ollama.chat(model="EEVE-korean-10.8B", messages=messages)
        state["hint_count"] += 1

         # 힌트 메시지 추가 방식 변경
        history.append({"role": "user", "content": "💡 힌트 요청"})  # 사용자 메시지
        history.append({"role": "assistant", "content": f"💡 힌트: {res['message']['content'].strip()}"})  # 어시스턴트 메시지

        return history, "", MAX_QUESTIONS - len(state["question_history"]), MAX_HINTS - state["hint_count"], idx,""
       
    # 정답 제출 처리
    if user_answer:
        if state["used_answer"]:
            return history, "⚠️ 이미 정답을 제출했습니다. 게임이 종료되었습니다.", 0, MAX_HINTS - state["hint_count"], idx,""

        state["used_answer"] = True
        judge_prompt = (
            "너는 사용자가 제출한 정답과 실제 정답이 '의미적으로 동일한지' 판단하는 역할이야.  "
            "'핵심 원인(왜 그런 일이 벌어졌는가)'만 비교하고, 세부적 표현은 무시해야해.  "
            " 다만, 사용자의 답변이 문제 상황과 명백히 무관하거나, 정답과 관련된 '의미 있는 원인'이 포함되지 않았다면 오답으로 판단해야 해. "
            
            "### 판단 기준 (반드시 준수)###  "
            "1. 정답 조건: 제출된 답변의 핵심 '사건 발생 이유'가 실제 정답과 본질적으로 같거나 유사할 때 "
            "2. 오답 조건: 사건의 원인이 다르거나, 핵심 사실이 누락되었을 때  "
            "3. 절대적 원칙: 시간 순서, 표현 방식, 수식어는 '전혀 고려하지 않음' "
            "   '동의어나 유추 가능한 간접적인 표현'도 같은 의미면 정답  "
            "단, 핵심 원인 자체가 빠지거나 의미 없는 내용이면 반드시 '오답' "
            "특히, '사과', '모른다', '죄송', '미안', '아무 말도 아닌 말장난' 등은 무조건 오답으로 처리해야 해.\n"
            
            "### 출력 원칙###  "
            " 반드시 '정답' 또는 '오답' 중 1개만 출력."
            " 그 외 어떤 설명도 절대 추가하면 안됩니다."
        )
        messages = [
            {"role": "system", "content": judge_prompt},
            {"role": "user", "content": f"정답: {truth.strip()}\n제출된 정답: {user_answer.strip()}"}
        ]
        res = ollama.chat(model="EEVE-korean-10.8B", messages=messages)
        result = res["message"]["content"].strip()

        history.append({"role": "user", "content": f"✅ 정답 제출: {user_answer}"})

        if "정답" in result:
            history.append({"role": "assistant", "content": f"정답입니다!\n{truth.strip()}"})
        else:
            history.append({"role": "assistant", "content": f"오답입니다. 정답은 다음과 같습니다:\n{truth.strip()}"})

        return history, "", 0, MAX_HINTS - state["hint_count"], idx,""

    if not user_question.strip() and not user_answer.strip() and not request_hint:
    # 아무 입력도 없으면 그대로 반환 (질문 입력란도 비움)
        return history, "", MAX_QUESTIONS - len(state["question_history"]), MAX_HINTS - state["hint_count"], idx, ""
    
    # 질문 횟수 체크
    if len(state["question_history"]) >= MAX_QUESTIONS:
        #return history, "질문 횟수를 모두 소진했습니다. 정답을 제출해보세요.", 0, MAX_HINTS - state["hint_count"], idx
        if user_question.strip():
            history.append({"role": "user", "content": user_question})
            history.append({"role": "assistant", "content": "질문 횟수를 모두 소진했습니다. 정답을 제출해보세요."})
        return history, "", 0, MAX_HINTS - state["hint_count"], idx,""
        
    # 일반 질문 처리
    qa_prompt = (
        "너는 추리 게임의 출제자이다. 사용자가 시나리오의 진실(정답)을 추리할 수 있도록 '예', '아니오', '예/아니오로 대답할 수 없음', '추리랑 관련된 질문이 아닙니다' 중 하나로만 답해야 한다.\n"

        "### 판단 기준 (반드시 준수)###  "
        "1. 사용자의 질문이 정답(진실)을 구성하는 사실 중 1개 이상과 의미적으로 같거나, 사실상 같은 내용을 묻는다면 반드시 '예'로 답해.\n"
        "   - 질문 어투, 말투, 단어, 높임말, 순서, 조사, 어순, 어는 무시하고 오직 의미 중심으로 판단해.\n"
        "   -  예시: 정답을 구성하는 사실 중 하나가 'A는 팔이 없다'일 때, 'A는 박수를 칠 수 없나요?'도 같은 의미로 간주해 '예'로 대답해야 한다.\n"
        "   -  사용자의 질문이 정답의 사실로부터 일반적이고 자연스러운 상식적 추론을 통해 도달 가능한 의미라면 같은 의미로 간주해 '예'로 대답해야 한다.\n"
        "   - 예시: '아이의 키가 작나요?', '아이의 키가 작아 ?', '애의 키가 작지?','아이 키가 작죠?','키가 작습니까?' 모두 같은 의미\n"
        "   - 또한, 정답을 구성하는 핵심 사실과의 직접적인 의미 연관성을 묻는 질문(예: '아이의 키가 작다'라는 사실이 있을 때 '정답이 아이의 키랑 관련있나요?','아이의 키가 중요한가요?')에서도, 관련이 있다고 파악되면 '예'로 답해."
        
        "2. 사용자의 질문이 정답을 구성하는 사실과 명백히 무관하거나, 반대 의미거나 틀린 해석이면 '아니오'로 답해.\n"
        "   - 핵심 사실과 관련 없어 보이는 주변 정보만 묻는 질문도 '아니오'\n"
        
        "3. '무엇', '왜', '언제', '어디서', '어떻게' 등 5W1H 계열 질문과 같이 예/아니오로 명확히 답할 수 없는 질문(예: '왜 그런 일이 일어났나요?')만 '예/아니오로 대답할 수 없음'으로 답해.\n"
        " -  5W1H 계열 질문은 원칙적으로 '예/아니오로 대답할 수 없음'이지만, 질문이 정답의 핵심 사실(예: 장소, 인물, 도구, 동기 등)을 간접적으로 지목하는 경우에는 반드시 의미 중심으로 판단해야 함."
        " - 예시: '어디서 일어난 사건인지가 중요한가요?' → 정답이 특정 장소에서 일어난 사건에 기반한 것이라면, '예'로 답해야 함."
        
        "4. 질문이 게임 진행과 전혀 관련 없으면 '추리랑 관련된 질문이 아닙니다.'로 답해.\n"
        "- 일상/잡담/명령 (예: '오늘 날씨 어때?','메뉴 추천 좀','힌트 줘')"
        "- 감탄사/의성어/말장난/무의미한 표현 : (예: '음','...','아아','와','유유','야야','ㅇ?','ㅇㅇ')"
        "- 정답을 무작정 요구하는 질문(예:'정답이 뭐야?','정답은?','정답 알려줘','답 내놔','정답을 알려줄 수 있나요?')"

        "### 출력 원칙###\n"
        "- 반드시 아래 네 가지 중 하나만 출력해야 한다:\n"
        "  '예', '아니오', '예/아니오로 대답할 수 없음', '추리랑 관련된 질문이 아닙니다'\n"
        "- 그 외에 추가 설명, 부연 설명, 이유는 금지한다.\n"
        "- 오직 지정해놓은 것 중 하나만 출력: '예' / '아니오' / '예/아니오로 대답할 수 없음' / '추리랑 관련된 질문이 아닙니다'\n"
    )
    
    messages = [
        {"role": "system", "content": qa_prompt},
        {"role": "user", "content": (
            f"[시나리오]\n{scenario.strip()}\n\n"
            f"[정답]\n{truth.strip()}\n\n"
            f"[지금까지 질문 목록]\n"
            + ("\n".join([f"Q: {msg['content']}" for msg in history if msg['role'] == 'user']) if history else "없음")
            + f"\n\n[새 질문]\n{user_question.strip()}\n"
        )}
    ]
    res = ollama.chat(model="EEVE-korean-10.8B", messages=messages)
    response = res['message']['content'].strip()

    for prefix in ["답변:", "A:", "Q:", "정답:", "오답:", "답:", "답 :"]:
        if response.startswith(prefix):
            response = response[len(prefix):].strip()

    state["question_history"].append(user_question)
    history.append({"role": "user", "content": user_question})
    history.append({"role": "assistant", "content": response})

    return history, "", MAX_QUESTIONS - len(state["question_history"]), MAX_HINTS - state["hint_count"], idx, ""

with gr.Blocks(css="""
.gradio-container {
    background: url('/gradio_api/file=img/beach.png') no-repeat center center fixed;
    background-size: cover;
    min-height: 100vh;
}
#center_wrap {
    display: flex;
    justify-content: center;
    align-items: center;
    height: 100vh;
}
#scenario-title {
    display: flex;
    flex-direction: column;
    gap: 24px;
    margin-top: 20px;
    align-items: center;
    min-width: 400px;
    max-width: 600px; 
}
#main_col {
    min-width: 400px;
    max-width: 600px;
    margin: auto;
    align-items: center;
    background: #f2f2f2;           /* 연한 회색 배경 */
    border-radius: 18px;           /* 둥근 모서리 */
    box-shadow: 0 4px 16px rgba(0,0,0,0.08); /* 그림자 */
    padding: 32px 24px 24px 24px;  /* 내부 여백 */
}
""") as demo:
    scenario_idx_state = gr.State(0)
    # --- 시나리오 선택 화면 ---
    with gr.Column(visible=True) as home_page:
        with gr.Row():
            gr.Column(scale=1)  # 왼쪽 여백
            with gr.Column(scale=2, elem_id="main_col"):  # 중앙 컨텐츠 (최대 600px)
                gr.Markdown("## 🐢 시나리오를 선택하세요!",elem_id="scenario-title")
                scenario_btns = []
                for idx, s in enumerate(scenarios):
                    btn = gr.Button(s["title"])
                    scenario_btns.append(btn)
            gr.Column(scale=1)  # 오른쪽 여백
    
   # --- 챗 봇 화면 ---
    with gr.Column(visible=False) as game_page:
        # 전체 행 구조
        with gr.Row():
            # 왼쪽 여백 (scale=1)
            gr.Column(scale=1)
            
            # 메인 컨텐츠 영역 (scale=2)
            with gr.Column(scale=2, min_width=500, elem_id="main_col"):
                # 질문 수/힌트 수 행
                with gr.Row():
                    remaining_text = gr.Textbox(label="남은 질문 수", interactive=False)
                    hint_text = gr.Textbox(label="남은 힌트 수", interactive=False)
                    
                # 챗봇 영역
                gr.Markdown("## 🐢 바다거북 수프 추리 게임")
                chatbox = gr.Chatbot(type="messages", layout="bubble", height=500)
                question_input = gr.Textbox(label="👉 질문을 입력해주세요")
                answer_input = gr.Textbox(label="✅ 정답 제출 (1회만)")
                
                # 버튼 행
                with gr.Row():
                    submit_btn = gr.Button("🎯 질문 / 정답 제출")
                    hint_btn = gr.Button("💡 힌트 요청")
                    home_btn = gr.Button("🏠 홈으로 가기")
            
            # 오른쪽 여백 (scale=1)
            gr.Column(scale=1)


    # --- 이벤트 핸들링 ---
    def show_game_ui(idx):
        scenario_states[idx] = get_empty_state()
        scenario_text = scenarios[idx]["scenario"].strip()
        history = [{"role": "assistant", "content": scenario_text}]
         # 튜토리얼이면 첫 안내 추가
        if idx == 0:
            first_q = tutorial_questions[0]
            history.append({"role": "assistant", "content": f"\n\n이렇게 질문해 보세요! 1️⃣ {first_q}"})
            scenario_states[idx]["tutorial_step"] = 0
        return (
            gr.update(visible=False),
            gr.update(visible=True),
            history, "", MAX_QUESTIONS, MAX_HINTS, idx,"")

    def go_home(idx):
        return (
            gr.update(visible=True),
            gr.update(visible=False),
            [], "", MAX_QUESTIONS, MAX_HINTS, 0,"")

    for idx, btn in enumerate(scenario_btns):
        btn.click(
            lambda idx=idx: show_game_ui(idx),
            inputs=None, 
            outputs=[home_page, game_page, chatbox, answer_input, remaining_text, hint_text, scenario_idx_state, question_input])

    # 홈 버튼 클릭
    home_btn.click(go_home, inputs=[scenario_idx_state], outputs=[home_page, game_page, chatbox, answer_input, remaining_text, hint_text, scenario_idx_state,question_input])
    
    # 질문/정답 제출 버튼 클릭
    submit_btn.click(handle_question, [question_input, answer_input, gr.State(False), chatbox, scenario_idx_state], [chatbox, answer_input, remaining_text, hint_text, scenario_idx_state,question_input])
    
    # 질문 입력칸에서 엔터
    question_input.submit(handle_question, [question_input, answer_input, gr.State(False), chatbox, scenario_idx_state], [chatbox, answer_input, remaining_text, hint_text, scenario_idx_state, question_input])
    
    # 정답 입력칸에서 엔터
    answer_input.submit(handle_question, [question_input, answer_input, gr.State(False), chatbox, scenario_idx_state], [chatbox, answer_input, remaining_text, hint_text, scenario_idx_state,question_input])
    
    # 힌트 버튼
    hint_btn.click(lambda chatbox, idx: handle_question("", "", True, chatbox, idx), [chatbox, scenario_idx_state], [chatbox, answer_input, remaining_text, hint_text, scenario_idx_state,question_input])

if __name__ == "__main__":
    demo.launch(share=True,allowed_paths=["img"])

* Running on local URL:  http://127.0.0.1:7893
* Running on public URL: https://ef97c6ea2c11a1dda3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
!pip install gradio ollama


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
